# Motor Neuron Segmentation

Motor neuron nuclei are labelled with antibodies against ISL1 and they are actually the cells of interest.
Although it would be nice to segment all nuclei, the primary objectivo is to find the motor neurons.

Let's select an image and then crop out a small piece of it to make testing easier.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from czifile import CziFile
from napari import Viewer
from scipy.ndimage import distance_transform_edt

In [ ]:
DATA_DIR = Path("../data/NS#1_Healhty-DAPI, WGA, ISL1, G3BP.czi")
image_handle = CziFile(DATA_DIR)
img = np.squeeze(image_handle.asarray())[2, :, 1000:1500, 1000:1500]

In [ ]:
scale = {
    values_dict["Id"]: values_dict["Value"] * 10**6
    for values_dict in image_handle.metadata(raw=False)["ImageDocument"]["Metadata"][
        "Scaling"
    ]["Items"]["Distance"]
}
spacing = (scale["Z"], scale["Y"], scale["X"])

## Cellpose

Let's begin by trying out cellpose.

In [ ]:
from cellpose import models  # noqa
from cellpose.io import logger_setup  # noqa
from skimage.morphology import remove_small_objects  # noqa

logger_setup()

In [ ]:
model = models.Cellpose(model_type="cyto3")

Estimate anisotropy: Z to X ratio

In [ ]:
anisotropy = scale["Z"] / scale["X"]
print(anisotropy)

In [ ]:
diameter = 10 / scale["X"]
print(diameter)

Estimated diameter of a cell is 8 to 14 $\mu$m.

In [ ]:
masks, flows, styles, diams = model.eval(
    [img],
    batch_size=8,
    diameter=diameter,
    z_axis=0,
    do_3D=True,
    channels=[0, 0],
    # stitch_threshold=0.5,
    normalize=True,
    anisotropy=anisotropy,
    # flow_threshold=2,
    compute_masks=True,
)

In [ ]:
z = 10

plt.imshow(img[z])
plt.colorbar()
plt.show()
plt.imshow(masks[0][z])

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(masks[0], scale=spacing)

These parameters seem to have worked better.
Masks are quite noisy.

In [ ]:
label = remove_small_objects(masks[0], 200)

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(label, scale=spacing)

## StarDist

Maybe stardist might work.
Not tested yet.

## Clesperanto Voronoi Otsu

Maybe Clesperanto's Voronoi and Otsu labelling works well here.

In [ ]:
import pyclesperanto_prototype as cle  # noqa

cle.available_device_names()

In [ ]:
input_gpu = cle.push(img[3])
input_gpu

I was unable to make clesperanto work on the laptop.

## Thresholding

Let's go for typical thresholding techniques as nuclei might be separated enough to not need watershed or maybe a simple watershed.

In [ ]:
from skimage.feature import peak_local_max  # noqa
from skimage.filters import (  # noqa
    median,
    scharr,
    threshold_otsu,
    try_all_threshold,
)
from skimage.morphology import ball, binary_erosion, dilation, disk  # noqa
from skimage.segmentation import watershed  # noqa

Let's begin with some median filtering to enhance a bit the edges.
I'm using my own footprint.

And then use Otsu thresolding.

In [ ]:
pixel_resolution = 0.5 / scale["X"]

footprint = np.stack(
    [
        np.zeros_like(disk(pixel_resolution)),
        disk(pixel_resolution),
        np.zeros_like(disk(pixel_resolution)),
    ]
)
midpoint = footprint.shape[1] // 2
footprint[0, midpoint, midpoint] = 1
footprint[2, midpoint, midpoint] = 1

filtered_image = median(img, footprint=footprint)
# filtered_image = gaussian(img, sigma=(0.8, 2, 2))
threshold = threshold_otsu(filtered_image)
nuclei = filtered_image > threshold

In [ ]:
try_all_threshold(img[10])

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_image(filtered_image, scale=spacing, visible=False)
viewer.add_labels(nuclei, scale=spacing)

Let's apply watershed to get a better separation between nuclei.
We will need the seeds.

In [ ]:
footprint = np.stack([np.zeros_like(disk(1)), disk(1), np.zeros_like(disk(1))])

inter = binary_erosion(nuclei, footprint=footprint)

transformed = distance_transform_edt(inter, sampling=spacing)

distance_between_nuclei = 1.7  # in um
pixel_min_distance = np.floor(distance_between_nuclei / scale["X"]).astype(int)

maxima = peak_local_max(transformed, min_distance=pixel_min_distance)

In [ ]:
print(pixel_min_distance, len(maxima))

We need an image for the edges of the nuclei to limit the watershed.

In [ ]:
edges = scharr(filtered_image)

In [ ]:
viewer = Viewer()
viewer.add_image(transformed, scale=spacing)
viewer.add_image(edges, scale=spacing)
viewer.add_points(
    maxima,
    name="points",
    scale=spacing,
    size=40,
    n_dimensional=True,  # points have 3D "extent"
    face_color="red",
)

In [ ]:
markers = np.zeros(img.shape, dtype=np.uint32)
marker_indices = tuple(np.round(maxima).astype(int).T)
markers[marker_indices] = np.arange(len(maxima)) + 1
markers = dilation(markers, ball(5))

segmented = watershed(
    -transformed,
    markers,
    mask=nuclei,
)

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(segmented, scale=spacing)

It's not the best and should be improved, but it's not that bad.